In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from tqdm import tqdm

# Inicjalizacja WebDriver
driver = webdriver.Chrome()

# Ustawienie ciasteczka języka na angielski
driver.get("https://steamcommunity.com/market")
driver.add_cookie({"name": "Steam_Language", "value": "english"})
time.sleep(1)  # Krótkie opóźnienie, aby upewnić się, że ciasteczko zostało ustawione

# Funkcja do pobrania nazw skrzynek z danej strony
def get_case_names(page_number):
    # skrzynki:
    url = f"https://steamcommunity.com/market/search?category_730_Type%5B%5D=tag_CSGO_Type_WeaponCase&appid=730&l=english#p{page_number}_popular_desc"
    #naklejki:
    # url = f"https://steamcommunity.com/market/search?category_730_ItemSet%5B%5D=any&category_730_ProPlayer%5B%5D=any&category_730_StickerCapsule%5B%5D=any&category_730_Tournament%5B%5D=any&category_730_TournamentTeam%5B%5D=any&category_730_Type%5B%5D=tag_CSGO_Tool_Sticker&category_730_Weapon%5B%5D=any&appid=730&l=english#p{page_number}_popular_desc"
    #karabin
    # url = f"https://steamcommunity.com/market/search?q=&category_730_ItemSet%5B%5D=any&category_730_ProPlayer%5B%5D=any&category_730_StickerCapsule%5B%5D=any&category_730_Tournament%5B%5D=any&category_730_TournamentTeam%5B%5D=any&category_730_Type%5B%5D=tag_CSGO_Type_Rifle&category_730_Weapon%5B%5D=any&appid=730#p{page_number}_popular_desc"
    #snajperki
    # url = f"https://steamcommunity.com/market/search?q=&category_730_ItemSet%5B%5D=any&category_730_ProPlayer%5B%5D=any&category_730_StickerCapsule%5B%5D=any&category_730_Tournament%5B%5D=any&category_730_TournamentTeam%5B%5D=any&category_730_Type%5B%5D=tag_CSGO_Type_SniperRifle&category_730_Weapon%5B%5D=any&appid=730#p{page_number}_popular_desc"
    #pistolety
    # url = f"https://steamcommunity.com/market/search?category_730_ItemSet%5B%5D=any&category_730_ProPlayer%5B%5D=any&category_730_StickerCapsule%5B%5D=any&category_730_Tournament%5B%5D=any&category_730_TournamentTeam%5B%5D=any&category_730_Type%5B%5D=tag_CSGO_Type_Pistol&category_730_Weapon%5B%5D=any&appid=730#p{page_number}_popular_desc"

    
    driver.get(url)
    time.sleep(2)  # Czekamy na załadowanie strony
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    case_names = []
    for item in soup.find_all("span", class_="market_listing_item_name"):
        case_names.append(item.text.strip())
    
    return case_names

# Pobieranie nazw skrzynek z wielu stron
all_case_names = []

#TODO należy wpisać ile jest stron danego przedmiotu
for i in range(1, 10+1):
    print(f"Pobieranie stron {i}")
    case_names = get_case_names(i)
    while all(name in all_case_names[-20:] for name in case_names):
        print("błąd 429, czekam 5 min...")
        print(f"jak na razie pobrano: {len(all_case_names)}/{(i-1)*10}")
        for i in tqdm(range(5*60)):
            time.sleep(1)
        driver.refresh()
        case_names = get_case_names(i)
        
    print(len(case_names))
    all_case_names.extend(case_names)
    
    

for name in all_case_names:
    print(name)



driver.quit()


Pobieranie stron 1
10
Pobieranie stron 2
10
Pobieranie stron 3
10
Pobieranie stron 4
10
Pobieranie stron 5
10
Pobieranie stron 6
10
Pobieranie stron 7
10
Pobieranie stron 8
10
Pobieranie stron 9
10
Pobieranie stron 10
10
Copenhagen 2024 Mirage Souvenir Package
Berlin 2019 Overpass Souvenir Package
Stockholm 2021 Overpass Souvenir Package
Paris 2023 Anubis Souvenir Package
Stockholm 2021 Mirage Souvenir Package
Copenhagen 2024 Inferno Souvenir Package
Rio 2022 Vertigo Souvenir Package
Rio 2022 Mirage Souvenir Package
Paris 2023 Mirage Souvenir Package
Paris 2023 Inferno Souvenir Package
MLG Columbus 2016 Mirage Souvenir Package
Atlanta 2017 Nuke Souvenir Package
Krakow 2017 Overpass Souvenir Package
Atlanta 2017 Overpass Souvenir Package
Paris 2023 Vertigo Souvenir Package
Paris 2023 Nuke Souvenir Package
Paris 2023 Mirage Souvenir Package
Stockholm 2021 Mirage Souvenir Package
Boston 2018 Inferno Souvenir Package
Copenhagen 2024 Inferno Souvenir Package
Cologne 2016 Mirage Souvenir Pac

In [20]:
import pickle as pkl

# Zapisz do pliku
with open('case_names.pkl', 'wb') as f:
    pkl.dump(all_case_names, f)

In [21]:
import pickle as pkl

# Załaduj z pliku
with open('case_names.pkl', 'rb') as f:
    all_case_names = pkl.load(f)

print(len(set(all_case_names)))
print(len(all_case_names))

74
100


In [22]:
import requests
import time
from datetime import datetime
from tqdm import tqdm


# Funkcja do pobierania ceny pojemnika z API Steam
def fetch_container_price(appid, market_hash_name):
    url = "https://steamcommunity.com/market/pricehistory/"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-US,en;q=0.9",
    }
    
    # Pliki cookie uzyskane po zalogowaniu się na konto Steam w przeglądarce
    cookies = {
        "steamLoginSecure": "", #enter cookie from your browser
        "sessionid": "" #enter cookie from your browser
    }
    
    params = {
        "appid": appid,
        "market_hash_name": market_hash_name
    }

    with requests.Session() as session:
        session.headers.update(headers)
        session.cookies.update(cookies)
        response = session.get(url, params=params)

        if response.status_code == 200:
            data = response.json()
            if data.get("success"):
                price = data['prices']
                return price
        elif response.status_code == 429:
            while response.status_code == 429:
                print("błąd 429, czekam 5 min...")
                for i in tqdm(range(5*60)):
                    time.sleep(1)
                response = session.get(url, params=params)
            data = response.json()
            if data.get("success"):
                price = data['prices']
                return price
        else:
            print(f'błąd: {response.status_code}')
    return None

# Lista nazw pojemników (należy zastąpić rzeczywistymi nazwami)
containers = list(set(all_case_names))
print(f"liczba przedmiotów: {len(containers)}")

case_listing = []
i=0
for container in containers:
    i+=1
    print(i)
    price = fetch_container_price(appid=730, market_hash_name=container)
    if price is not None:
        case_listing.append([container] + price)
        print(f"Cena produktu '{container}': {price[-1][0]} {price[-1][1]}")
    else:
        print(f"Nie udało się pobrać ceny dla '{container}'")
        

liczba przedmiotów: 74
1
Cena produktu 'DreamHack Cluj-Napoca 2015 Train Souvenir Package': Mar 03 2025 23: +0 160.265
2
Cena produktu 'Rio 2022 Mirage Souvenir Package': Mar 10 2025 09: +0 14.14
3
Cena produktu 'Shanghai 2024 Dust II Souvenir Package': Mar 10 2025 16: +0 12.968
4
Cena produktu 'Copenhagen 2024 Overpass Souvenir Package': Mar 10 2025 15: +0 9.419
5
Cena produktu 'Stockholm 2021 Vertigo Souvenir Package': Mar 10 2025 13: +0 18.67
6
Cena produktu 'EMS Katowice 2014 Legends': Jun 28 2023 01: +0 6929.378
7
Cena produktu 'Copenhagen 2024 Inferno Souvenir Package': Mar 10 2025 14: +0 10.104
8
Cena produktu 'Rio 2022 Overpass Souvenir Package': Mar 10 2025 13: +0 18.992
9
Cena produktu 'DreamHack Cluj-Napoca 2015 Mirage Souvenir Package': Feb 27 2025 13: +0 173.48
10
Cena produktu 'Katowice 2019 Train Souvenir Package': Feb 26 2025 22: +0 56.033
11
Cena produktu 'Berlin 2019 Overpass Souvenir Package': Mar 09 2025 21: +0 64.214
12
Cena produktu 'ESL One Katowice 2015 Mirage S

In [23]:
# test if one of product is downloaded correctly
#TODO set the right name
# fetch_container_price(appid=730, market_hash_name='AUG | Midnight Lily (Well-Worn)')

In [24]:
import pandas as pd
from datetime import datetime
import numpy as np

today = pd.to_datetime(datetime.now().date())
meta_data_raw = {}
meta_data_prim = {}

for df in case_listing:
    name = df[0]
    data = pd.DataFrame(df[1:], columns = ['date', 'price', 'volume'])
    data['volume'] = data['volume'].fillna(0).astype(int)
    data['date'] = pd.to_datetime(data['date'], format='%b %d %Y %H: +0')
    data['days_diff'] = (today - data['date']).dt.days
    years = int(data.iloc[0,-1]/365)
    data_2 = []
    for y in range(years):
        interval = (data['days_diff'] > y * 365) & (data['days_diff'] <= (y + 1) * 365)
        if np.sum(interval) == 0:
            data_2.append([0,0])
        else:
            data_2.append([np.median(data['price'][interval]), np.sum(data['volume'][interval])]) 
    data_2 = pd.DataFrame(data_2, columns = ['price', 'volume'])
    meta_data_raw[name] = data_2
    meta_data_prim[name] = (data_2.iloc[:-1,:].reset_index(drop=True) - data_2.iloc[1:,:].reset_index(drop=True)) / data_2.iloc[1:,:].reset_index(drop=True)

In [25]:
len(meta_data_raw)

74

In [26]:
price = []

for name in meta_data_prim:
    if len(meta_data_prim[name]) != 0:
        price.append([name, meta_data_raw[name]['price'][0],
                      meta_data_raw[name]['volume'][0], meta_data_prim[name]['price'][0],
                      np.mean(meta_data_prim[name]['price']), np.mean(meta_data_prim[name]['volume'])])
price = pd.DataFrame(price, columns = ['name', 'price', 'last_volume', 'last_price', 'mean_price', 'mean_volume'])
price['abs_vol'] = np.abs(price['mean_volume'])
price.to_csv('price.csv', index=False)

In [27]:
import pandas as pd
import numpy as np

price = pd.read_csv('price.csv')
price.head()

,name,price,last_volume,last_price,mean_price,mean_volume,abs_vol
0,DreamHack Cluj-Napoca 2015 Train Souvenir Package,109.1770,40,0.177745,0.345510,-0.306814,0.306814
1,Rio 2022 Mirage Souvenir Package,14.6090,9388,-0.055595,-0.055595,-0.783247,0.783247
2,Stockholm 2021 Vertigo Souvenir Package,16.5960,3450,-0.184352,0.242402,-0.660100,0.660100
3,EMS Katowice 2014 Legends,0.0000,0,-1.000000,inf,inf,inf
4,Rio 2022 Overpass Souvenir Package,13.6455,1948,-0.144697,-0.144697,-0.719510,0.719510


In [28]:
df_r = price.sort_values(by='last_price', ascending=False)
df_new = df_r[df_r['name'].str.contains(r'FAMAS', regex=True)]

In [29]:
pd.set_option('display.max_rows', None)  # Nieograniczona liczba wierszy
pd.set_option('display.max_columns', None)  # Nieograniczona liczba kolumn
pd.set_option('display.max_colwidth', None) # Nieograniczona szerokość kolumn

price.sort_values(by='last_price', ascending=False)
# df_new.sort_values(by='last_price', ascending=False)

,name,price,last_volume,last_price,mean_price,mean_volume,abs_vol
40,ESL One Cologne 2014 Nuke Souvenir Package,1676.2530,1,2.149957,0.546533,-0.268919,0.268919
37,ESL One Katowice 2015 Overpass Souvenir Package,1003.7670,7,0.587754,0.399967,-0.386385,0.386385
15,ESL One Cologne 2014 Mirage Souvenir Package,722.9450,6,0.566002,0.391314,-0.275365,0.275365
8,ESL One Katowice 2015 Mirage Souvenir Package,303.2890,37,0.414456,0.401730,-0.278221,0.278221
20,MLG Columbus 2016 Inferno Souvenir Package,159.6420,11,0.402090,0.423948,-0.297073,0.297073
36,MLG Columbus 2016 Mirage Souvenir Package,132.6970,52,0.378799,0.410038,-0.305478,0.305478
30,Katowice 2019 Mirage Souvenir Package,85.0450,72,0.338743,0.464370,-0.435148,0.435148
17,London 2018 Mirage Souvenir Package,54.6290,365,0.335183,0.432537,-0.374234,0.374234
29,Boston 2018 Inferno Souvenir Package,58.5530,251,0.322604,0.434558,-0.397411,0.397411
28,ESL One Katowice 2015 Cache Souvenir Package,250.1890,45,0.315324,0.331519,-0.319859,0.319859


In [30]:
"""
__kejsiki (396)__:
sortowane po last price (te które urosły w ostatnim roku [wskaźnik wydawał się być mało trafny])
Autograph Capsule | Vexed Gaming | Cluj-Napoca 2015 może wybijać ath
Autograph Capsule | Counter Logic Gaming | Cologne 2015 już po ath
Berlin 2019 Minor Challengers Autograph Capsule tanie ale całkiem nowe

sortowane po mean pricee
Glove Case         l 0.61 m: 3.65
Prisma 2 Case      l 0.55 m: 2.58
Gamma 2 Case       l 0.49 m: 1.78
Spectrum 2 Case 	l 0.57 m: 1.74

wychodzi na to że najbardziej opłacają się glove case'y 
spektrum też jest spoko. Dobrze drożeją pamiątki chodzące po 7k.

__naklejki (6690)__:
Sticker | dav1deuS | Rio 2022 jedyna sensowna jaką znalazłem narazie
reszta naklejek to jakieś wały są...
Naklejka | SmithZz | Londyn 2018
Sticker | FaZe | 2020 RMR

__karabiny (1452)__:
M4A4 | Global Offensive
AUG | Surveillance (Minimal Wear)
AUG | Surveillance (Battle-Scarred)
"""

"\n__kejsiki (396)__:\nsortowane po last price (te które urosły w ostatnim roku [wskaźnik wydawał się być mało trafny])\nAutograph Capsule | Vexed Gaming | Cluj-Napoca 2015 może wybijać ath\nAutograph Capsule | Counter Logic Gaming | Cologne 2015 już po ath\nBerlin 2019 Minor Challengers Autograph Capsule tanie ale całkiem nowe\n\nsortowane po mean pricee\nGlove Case         l 0.61 m: 3.65\nPrisma 2 Case      l 0.55 m: 2.58\nGamma 2 Case       l 0.49 m: 1.78\nSpectrum 2 Case \tl 0.57 m: 1.74\n\nwychodzi na to że najbardziej opłacają się glove case'y \nspektrum też jest spoko. Dobrze drożeją pamiątki chodzące po 7k.\n\n__naklejki (6690)__:\nSticker | dav1deuS | Rio 2022 jedyna sensowna jaką znalazłem narazie\nreszta naklejek to jakieś wały są...\nNaklejka | SmithZz | Londyn 2018\nSticker | FaZe | 2020 RMR\n\n__karabiny (1452)__:\nM4A4 | Global Offensive\nAUG | Surveillance (Minimal Wear)\nAUG | Surveillance (Battle-Scarred)\n"

In [31]:
# invested = ['Skrzynia Chromatyczności 2', 'Skrzynia Rękawic', 'Skrzynia Spektrum', 'Skrzynia Gamma 2', 'Pojemnik z autografem | Natus Vincere | Cluj-Napoca 2015',
#            'Skrzynia operacji Breakout', 'Zimowa walizka e-sportowa 2013', 'Skrzynia Łowcy', 'Skrzynia CS:GO 2', 'P250 | Rój (prosto z fabryki)',
#            'AK-47 | Kartel (po testach bojowych)', 'Galil AR | Łowiecka przykrywka (prosto z fabryki)', 'Galil AR | Szkarłatne tsunami (prosto z fabryki)',
#            'Pojemnik z autografem | Counter Logic Gaming | Kolonia 2016', 'M4A4 | Global Offensive (lekkie zużycie)']

invested = [
    'Chroma 2 Case', 'Glove Case', 'Spectrum Case', 'Gamma 2 Case', 
    'Autograph Capsule | Natus Vincere | Cluj-Napoca 2015',
    'Operation Breakout Case', 'Winter Offensive Weapon Case', 
    'Huntsman Weapon Case', 'CS:GO Weapon Case 2', 'P250 | Hive (Factory New)', 
    'AK-47 | Cartel (Field-Tested)', 'Galil AR | Hunting Blind (Factory New)', 
    'Galil AR | Crimson Tsunami (Factory New)', 
    'Autograph Capsule | Counter Logic Gaming | Cologne 2016', 
    'M4A4 | Global Offensive (Minimal Wear)'
]

In [32]:
import requests
import time
from datetime import datetime


# Funkcja do pobierania ceny pojemnika z API Steam
def fetch_container_price(appid, market_hash_name):
    url = "https://steamcommunity.com/market/pricehistory/"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-US,en;q=0.9",
    }
    
    # Pliki cookie uzyskane po zalogowaniu się na konto Steam w przeglądarce
    cookies = {
        "steamLoginSecure": "",
        "sessionid": ""
    }
        
    params = {
        "appid": appid,
        "market_hash_name": market_hash_name
    }

    with requests.Session() as session:
        session.headers.update(headers)
        session.cookies.update(cookies)
        response = session.get(url, params=params)

        if response.status_code == 200:
            data = response.json()
            if data.get("success"):
                price = data['prices']
                return price
        elif response.status_code == 429:
            while response.status_code == 429:
                print("błąd 429, czekam 5 min...")
                time.sleep(60*5)
                response = session.get(url, params=params)
            data = response.json()
            if data.get("success"):
                price = data['prices']
                return price
        else:
            print(f'błąd: {response.status_code}')
    return None

# Lista nazw pojemników (należy zastąpić rzeczywistymi nazwami)
containers = invested
print(f"liczba przedmiotów: {len(containers)}")

case_listing = []
i=0
for container in containers:
    i+=1
    print(i)
    price = fetch_container_price(appid=730, market_hash_name=container)
    if price is not None:
        case_listing.append([container] + price)
        print(f"Cena pojemnika '{container}': {price[-1][0]} {price[-1][1]}")
    else:
        print(f"Nie udało się pobrać ceny dla '{container}'")

liczba przedmiotów: 15
1
Cena pojemnika 'Chroma 2 Case': Mar 10 2025 16: +0 16.211
2
Cena pojemnika 'Glove Case': Mar 10 2025 15: +0 40.22
3
Cena pojemnika 'Spectrum Case': Mar 10 2025 16: +0 21.168
4
Cena pojemnika 'Gamma 2 Case': Mar 10 2025 16: +0 16.3
5
Cena pojemnika 'Autograph Capsule | Natus Vincere | Cluj-Napoca 2015': Mar 03 2025 15: +0 28.102
6
błąd: 500
Nie udało się pobrać ceny dla 'Operation Breakout Case'
7
Cena pojemnika 'Winter Offensive Weapon Case': Mar 10 2025 16: +0 34.345
8
Cena pojemnika 'Huntsman Weapon Case': Mar 10 2025 16: +0 53.212
9
Cena pojemnika 'CS:GO Weapon Case 2': Mar 10 2025 16: +0 59.567
10
Cena pojemnika 'P250 | Hive (Factory New)': Mar 10 2025 15: +0 16.789
11
Cena pojemnika 'AK-47 | Cartel (Field-Tested)': Mar 10 2025 16: +0 73.143
12
Cena pojemnika 'Galil AR | Hunting Blind (Factory New)': Mar 08 2025 20: +0 26.53
13
Cena pojemnika 'Galil AR | Crimson Tsunami (Factory New)': Mar 10 2025 16: +0 19.801
14
Cena pojemnika 'Autograph Capsule | Counter

In [33]:
import pandas as pd
from datetime import datetime
import numpy as np

today = pd.to_datetime(datetime.now().date())
meta_data_raw2 = {}
meta_data_prim2 = {}

for df in case_listing:
    name = df[0]
    data = pd.DataFrame(df[1:], columns = ['date', 'price', 'volume'])
    data['volume'] = data['volume'].fillna(0).astype(int)
    data['date'] = pd.to_datetime(data['date'], format='%b %d %Y %H: +0')
    data['days_diff'] = (today - data['date']).dt.days
    years = int(data.iloc[0,-1]/365)
    data_2 = []
    for y in range(years):
        interval = (data['days_diff'] > y * 365) & (data['days_diff'] <= (y + 1) * 365)
        if np.sum(interval) == 0:
            data_2.append([0,0])
        else:
            data_2.append([np.mean(data['price'][interval]), np.sum(data['volume'][interval])]) 
    data_2 = pd.DataFrame(data_2, columns = ['price', 'volume'])
    meta_data_raw2[name] = data_2
    meta_data_prim2[name] = (data_2.iloc[:-1,:].reset_index(drop=True) - data_2.iloc[1:,:].reset_index(drop=True)) / data_2.iloc[1:,:].reset_index(drop=True)

price2 = []

for name in meta_data_prim2:
    if len(meta_data_prim2[name]) != 0:
        price2.append([name, meta_data_raw2[name]['price'][0],
                      meta_data_raw2[name]['volume'][0], meta_data_prim2[name]['price'][0],
                      np.mean(meta_data_prim2[name]['price']), np.mean(meta_data_prim2[name]['volume'])])
price2 = pd.DataFrame(price2, columns = ['name', 'price', 'last_volume', 'last_price', 'mean_price', 'mean_volume'])
price2['abs_vol'] = np.abs(price2['mean_volume'])
price2.to_csv('inwestycje.csv', index=False)

In [34]:
pd.set_option('display.max_rows', None)  # Nieograniczona liczba wierszy
pd.set_option('display.max_columns', None)  # Nieograniczona liczba kolumn
pd.set_option('display.max_colwidth', None) # Nieograniczona szerokość kolumn

price2.sort_values(by='abs_vol', ascending=False)

,name,price,last_volume,last_price,mean_price,mean_volume,abs_vol
13,M4A4 | Global Offensive (Minimal Wear),115.533539,2079,0.801628,1.291649,-0.636756,0.636756
4,Autograph Capsule | Natus Vincere | Cluj-Napoca 2015,27.968000,223,0.428467,0.455803,-0.371906,0.371906
10,Galil AR | Hunting Blind (Factory New),27.813004,445,1.135368,0.726795,-0.301941,0.301941
8,P250 | Hive (Factory New),25.177191,2847,0.069965,0.409093,-0.291775,0.291775
9,AK-47 | Cartel (Field-Tested),60.076236,23961,0.809610,0.275396,-0.274713,0.274713
6,Huntsman Weapon Case,48.233990,109445,0.233169,0.508789,-0.265473,0.265473
5,Winter Offensive Weapon Case,32.396423,78010,0.213688,0.478134,-0.258415,0.258415
12,Autograph Capsule | Counter Logic Gaming | Cologne 2016,76.974269,29,0.320772,0.530086,-0.257861,0.257861
2,Spectrum Case,17.769504,1132583,0.823418,2.257608,-0.232469,0.232469
7,CS:GO Weapon Case 2,55.747911,68519,0.068653,0.706871,-0.214611,0.214611
